In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#!pip install dash
#!pip install dash-bootstrap-components
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import os
import plotly.graph_objects as go
from sklearn.decomposition import PCA
import numpy as np


# Definir o caminho para o diretório onde os arquivos Excel estão localizados
path = "/content/drive/MyDrive/preprocessed/"  # Atualize o caminho conforme necessário

# Função para ler as planilhas
def ler_dados(file):
    try:
        dados = pd.read_excel(file)
        dados = dados.iloc[:, 1:]  # Seleciona todas as colunas, exceto a primeira
        numeric_cols = dados.select_dtypes(include='number')

        # Padronizando os dados
        for col in numeric_cols.columns:
            media_col = numeric_cols[col].mean()
            desvio_padrao_col = numeric_cols[col].std()
            numeric_cols[col] = (numeric_cols[col] - media_col) / desvio_padrao_col

        # Calcular a matriz de correlação
        correlacao = numeric_cols.corr()
        return numeric_cols, correlacao
    except Exception as e:
        print(f"Erro ao ler o arquivo {file}: {e}")
        return pd.DataFrame(), pd.DataFrame()

# Função para obter as informações do poço
def obter_informacoes_poco(arquivo):
    info_pocos = pd.DataFrame({
        "Poço": ["3-BRSA-1148-SE", "3-BRSA-1098-SE", "3-BRSA-1093-SE", "3-SZ-464-SE", "3-BRSA-912-SE",
                 "3-BRSA-1279-SE", "3-BRSA-1262-SE", "3-BRSA-1307D-SE", "6-BRSA-1202D-BA", "1-BRSA-470D-BA",
                 "4-BRSA-422-BA", "3-BRSA-349-RN", "3-BRSA-1295-RN", "3-BRSA-1193-RN", "3-BRSA-1224-RN",
                 "3-BRSA-1210-RN", "3-BRSA-1325-RN", "3-BRSA-1320-RN", "3-BRSA-1341-RN", "3-OGX-95-MA",
                 "4-PGN-25-MA", "1-OGX-117-MA", "4-PGN-19-MA", "7-GVB-8-MA", "7-GVB-6-MA", "7-RUC-59DP-AM",
                 "3-BRSA-515-AM", "3-BRSA-1057D-AM", "3-BRSA-670-AM", "3-BRSA-1121-AM","3-BRSA-1057D"],
        "Campo": ["Siririzinho", "Siririzinho", "Siririzinho", "Siririzinho", "Carmópolis",
                  "Carmópolis", "Carmópolis", "Carmópolis", "Dom João", "Guanambi",
                  "CEXIS", "Canto do Amaro", "Canto do Amaro", "Canto do Amaro", "Estreito",
                  "Estreito", "Estreito", "Alto do Rodrigues", "Alto do Rodrigues", "Gavião Branco",
                  "Gavião Tesoura", "Gavião Tesoura", "Gavião Tesoura", "Gavião Branco", "Gavião Branco",
                  "Rio Urucu", "Rio Urucu", "Rio Urucu", "Rio Urucu", "Rio Urucu","NA"],
        "Bacia": ["Sergipe", "Sergipe", "Sergipe", "Sergipe", "Sergipe",
                  "Sergipe", "Sergipe", "Sergipe", "Recôncavo", "Recôncavo",
                  "Recôncavo", "Potiguar", "Potiguar", "Potiguar", "Potiguar",
                  "Potiguar", "Potiguar", "Potiguar", "Potiguar", "Parnaíba",
                  "Parnaíba", "Parnaíba", "Parnaíba", "Parnaíba", "Parnaíba",
                  "Solimões", "Solimões", "Solimões", "Solimões", "Solimões","NA"]
    })

    info = info_pocos[info_pocos["Poço"] == arquivo.replace(".xlsx", "")]
    return info

# Layout da aplicação
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1(" Séries Temporais dos Poços com PCA", className="text-center text-primary mb-4"), width=12)
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Selecione o poço:"),
            dcc.Dropdown(
                id='arquivo-dropdown',
                options=[{'label': f, 'value': f} for f in os.listdir(path) if f.endswith('.xlsx')],
                value=None
            ),
            dbc.Button("Atualizar", id="atualizar-btn", color="success", className="mt-3")
        ], width=4),
        dbc.Col([dcc.Graph(id='grafico-pca1')], width=8)
    ])
])

# Callback para atualizar o gráfico de PCA1
@app.callback(
    Output('grafico-pca1', 'figure'),
    [Input('atualizar-btn', 'n_clicks')],
    [State('arquivo-dropdown', 'value')]
)
def atualizar_grafico_pca1(n_clicks, arquivo):
    if not arquivo:
        return {}

    # Carregar dados e calcular PCA1
    file_path = os.path.join(path, arquivo)
    dados = ler_dados_e_pca(file_path)

    # Verificar se os dados e o PCA1 foram calculados corretamente
    if 'PCA1' not in dados.columns:
        return {}

    # Criar gráfico de linhas para o PCA1
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dados['PROF (m)'],  # Usar profundidade como eixo X
        y=dados['PCA1'],      # Usar PCA1 como eixo Y
        mode='lines',
        name='PCA1',
        line=dict(color='blue', width=2)
    ))

    fig.update_layout(
        title=f"Gráfico de PCA1 ao Longo da Profundidade ({arquivo})",
        xaxis_title="Profundidade (m)",
        yaxis_title="PCA1",
        template="plotly_white"
    )

    return fig

# Rodar a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>